In [11]:
import re
import pandas as pd
from bokeh.plotting import figure, show, output_file
from bokeh.models import Select, ColumnDataSource, CustomJS
from bokeh.layouts import column
from bokeh.models import Span
from bokeh.palettes import Category10
from bokeh.models import HoverTool

import matplotlib.pyplot as plt
import datetime as dt

In [12]:
fred_input_folder = '/zine/HPC02S1/ex-dveloza/AGORA/apps/fred_colombia_implementation/fred_input_files'
synthetic_pop_data_folder = '/zine/HPC02S1/ex-dveloza/AGORA/apps/synthetic_populations/data'

In [13]:
df_mun_info = pd.read_csv(f'{synthetic_pop_data_folder}/processed_data/geodata/geoinfo_municipios_colombia.csv')[['COD_DEPTO','NOM_DEPART']].drop_duplicates()
dept_cods = df_mun_info.COD_DEPTO.sort_values()[:-1]

In [14]:
dataframes_mov = {}
dataframes_contacts = {}
for dept_code in dept_cods:
    mobility_file_path  = f'{fred_input_folder}/mobility_trends/{dept_code}_mobility_trends.csv'
    contacts_file_path   = f'{fred_input_folder}/contacts/{dept_code}_contacts.csv'

    df_mobility = pd.read_csv(mobility_file_path)
    df_contacts  = pd.read_csv(contacts_file_path)

    df_mobility['date'] = pd.to_datetime(df_mobility['date'])
    df_contacts['Date']  = pd.to_datetime(df_contacts['Date'])

    df_mobility['mobility_trend'] = 1 + df_mobility['mobility_trend']

    dept_name = df_mun_info[df_mun_info.COD_DEPTO == dept_code].NOM_DEPART.tolist()[0]

    dataframes_mov[dept_name]       = df_mobility
    dataframes_contacts[dept_name]   = df_contacts

In [15]:
df_contacts

,Date,day,Contacts_total,Contacts_Prop
0,2020-10-09,282,1.137059,1.000000
1,2020-10-10,283,1.147559,1.009234
2,2020-10-11,284,1.158281,1.018664
3,2020-10-12,285,1.175739,1.034018
4,2020-10-13,286,1.191921,1.048249
...,...,...,...,...
151,2021-03-09,433,1.687192,1.483822
152,2021-03-10,434,1.692855,1.488801
153,2021-03-11,435,1.700300,1.495350
154,2021-03-12,436,1.708922,1.502932


In [16]:
# Convert DataFrames to ColumnDataSource for mobility and contacts
sources_mov = {name: ColumnDataSource(df) for name, df in dataframes_mov.items()}
sources_contacts = {name: ColumnDataSource(df) for name, df in dataframes_contacts.items()}

# Initial ColumnDataSource for mobility and contacts
initial_source_mov = dataframes_mov['CUNDINAMARCA']
initial_source_contacts = dataframes_contacts['CUNDINAMARCA']

# Create the figure with datetime x-axis
p = figure(x_axis_type='datetime', title="Mobility and contacts Trends Over Time", width=800, height=400)

# Add lines for mobility and contacts
mobility_line = p.line(x='date', y='mobility_trend', source=initial_source_mov, line_width=2, color='blue', legend_label='Mobility Trend')
contacts_line = p.line(x='Date', y='Contacts_total', source=initial_source_contacts, line_width=2, color='green', legend_label='contacts Trend')

# Add hover tool
p.add_tools(HoverTool(tooltips=[('Date', '@Date{%F}'), ('Trend', '@Contacts_total')], formatters={'@date': 'datetime'}))

# Adding a horizontal line at y = 1
hline = Span(location=1, dimension='width', line_color='red', line_width=3)
p.add_layout(hline)

# Set legend click policy
p.legend.click_policy = "hide"

# y axis
p.y_range.start = 0.05
p.y_range.end = 2

# Select widget with JavaScript callback for changing data source
select = Select(title="Choose Department:", value='CUNDINAMARCA', options=list(sources_mov.keys()))

select.js_on_change('value', CustomJS(args=dict(sources_mov=sources_mov, sources_contacts=sources_contacts, mobility_line=mobility_line, contacts_line=contacts_line), code="""
    const selected = cb_obj.value;
    const new_source_mov = sources_mov[selected];
    const new_source_contacts = sources_contacts[selected];

    // Update data source for mobility and contacts lines
    mobility_line.data_source.data = new_source_mov.data;
    contacts_line.data_source.data = new_source_contacts.data;
    
    // Emit the change to update the plot
    mobility_line.data_source.change.emit();
    contacts_line.data_source.change.emit();
"""))

# Layout
layout = column(select, p)

# Output to HTML
output_file('../output_files/mobility_contacts_trends.html')
show(layout)